# Performance by product --> SoM Dashboard

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Brands/SoM/data/'
CPG = 'danone' 
nielsen_cat = True #True JUST FOR RB_HYHO, DANONE, UNILEVER SO FAR!!!
past = False
SOM_CPG = CPG.title()

## Import

In [2]:
import os
import numpy as np
import pandas as pd
from IPython.display import display

pd.options.display.float_format = '{:.2f}'.format
print([x for x in os.listdir(BASE_DIR) if x.endswith(f'{CPG}.csv')])

['som_danone.csv', 'nielsen_danone.csv', 'products_danone.csv', 'nielsen_pred_danone.csv']


In [3]:
df = pd.read_csv(os.path.join(BASE_DIR,f'products_{CPG}.csv'),
                 sep='\t', encoding='UTF-16', low_memory=False)
df.sample(2)

,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,sales,orders,avg ticket,users,units found,units replaced,units requested,found rate,fulfillment,frequency
8671,noviembre 2021,MX,León,Chedraui,Yogurt,24,1149858,Yoghurt sabor manzana,{7501032398415},Bote 900 g,...,28,1,28,1,1,0,1,100.0%,100.0%,1.00
16376,noviembre 2021,MX,Toluca,Chedraui,Water,35,323924,Agua de coco orgánica,{7503006575785},1 l,...,158,1,158,1,4,0,4,100.0%,100.0%,1.00


In [4]:
# df.rename(columns={'month  / year':'month  ','sales [currency]':'sales'}, inplace=True)

In [5]:
aux_ini = df.copy()
aux_ini['sales'] = aux_ini['sales'].astype(str).str.replace(',','').astype(float)
aux_ini = aux_ini.groupby('month  ')['sales'].sum().reset_index()
aux_ini

,month,sales
0,noviembre 2021,16293391.00


In [6]:
import re
nielsen = pd.read_csv(BASE_DIR + f'nielsen_{CPG}.csv',
                      sep='\t',encoding='utf-16'
                     ).drop_duplicates(subset=['UPC','PRODUCT ID'])
display(nielsen['UPC'].head())
nielsen['UPC'] = nielsen['UPC'].apply(lambda x: re.sub('[^\d]','',str(x)))
nielsen['UPC'] = nielsen['UPC'].astype(str).str.strip().str.split().str[0].fillna(0).astype(int)
display(nielsen['UPC'].head())
display(nielsen['UPC'].head())
nielsen.sample(2)

0    75009076
1    75009083
2    75009090
3    75010065
4    75010751
Name: UPC, dtype: object

0    75009076
1    75009083
2    75009090
3    75010065
4    75010751
Name: UPC, dtype: int64

0    75009076
1    75009083
2    75009090
3    75010065
4    75010751
Name: UPC, dtype: int64

,UPC,FORMATO,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,PRODUCT ID,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
5012,7501663000800,FAM,DEMAS FABRICANTES,DEMAS FABRICANTES,PENA REGIA,PENA REGIA,10.00,PENA REGIA AGUA NATURAL BOT.PLAST.10000ML NAL.,NaN,AGUA NATURAL,AGUA NATURAL,NaN,NaN
6145,7503010691624,FAM,DEMAS FABRICANTES,DEMAS FABRICANTES,ORANGINA,ORANGINA,3.79,ORANGINA BEBIDAS REFRESCANTES NARANJA 3.785 LT...,NaN,BEBIDAS SABORIZADAS,BEBIDAS REFRESCANTES,NaN,NaN


## Functions

In [7]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

In [8]:
def limpio(data,month_col,to_drop):
    df = data.copy()
    df[['mes','anio']] = df[month_col].str.split(expand=True)
    meses = dict(zip(['enero','febrero','marzo','abril','mayo','junio','julio',
                      'agosto','septiembre','octubre','noviembre','diciembre'],range(1,13)))
    df['mes'] = df['mes'].replace(meses)
    df.drop(columns=to_drop,inplace=True)
    df.columns = [str(x).replace('.','').replace(' ','_').lower() for x in df.columns]
    for col in ['brand_filter','cpg_filter']:
        df[f'{col[:-7]}_id'] = df[col].apply(lambda x: ','.join(re.findall('\s\((\d+)\)',str(x))))
        df[f'{col[:-7]}_name'] = df[col].apply(lambda x: re.sub('\s\(\d+\)','',str(x)))
    return df

## Transform

### Barcodes

In [9]:
df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')
df['barcodes'] = df['barcodes'].str.split(',',n=1).fillna('.').apply(lambda x:x[0])
df['barcodes'].sample(4)

/var/folders/j4/1p33_bh96yn8pdz3b_5t80hm0000gn/T/ipykernel_2430/2845833164.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')


3308      818290017741
4694     7501055900411
14647    7501055361205
1711     7501020563801
Name: barcodes, dtype: object

### Merge

In [10]:
df = df.astype(str).merge(nielsen.astype(str).rename(columns={'UPC':'barcodes',
                                                              'PRODUCT ID':'product id'}),
                          on=['barcodes'] if nielsen_cat else ['barcodes','product id'],
                          how='left' if nielsen_cat else 'left')
if nielsen_cat:
    df = df.rename(columns={'product id_x':'product_id'}).drop(columns='product id_y')

df.sample(2)

,month,country,city name,store_name,category en,category_id,product_id,product name,barcodes,package,...,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
9445,noviembre 2021,MX,Mérida,Chedraui,Plant-based Beverages,1516,3617625,Leche de coco sin azúcar,7501032399290,Cartón 946 ml,...,DANONE,nan,SILK,nan,0.946,SILK ALIMENTO LIQUIDO DE COCO S/AZUCAR CARTON ...,PLANT BASED,PLANT BASED,COCO,UHT
6734,noviembre 2021,MX,Guadalajara,Chedraui,Plant-based Beverages,1516,381931,Queso mozarella rallado,7500462286071,Bolsa 400 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### CPG

In [11]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
noviembre 2021,16293391.00,16293391.00,0.00,0.00


In [12]:
prod_cpg = pd.read_csv(os.path.join(BASE_DIR,'pbbc.csv'),
                       #sep='\t',encoding='utf-16'
                      )
prod_cpg.sample(4)

,n,product_id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name
83554,1,1248041,38629000750,981,Galletas dulces,5069,La Moderna,146,La Moderna pasado
54668,1,355837,7501058711670,990,Ambientadores y velas,337,Air Wick,11,Reckitt Benckiser MX - HyHo
32556,1,220490,7622210429292,999,Chicles y mentas,4241,Clorets,115,Mondelez
18199,1,177309,7506339394924,1582,Suavizantes,659,Downy,1,P&G MX


In [13]:
prod_cpg.drop_duplicates(('barcodes' if nielsen_cat else 'product_id'),inplace=True)
prod_cpg = prod_cpg.astype(str).rename(columns={'product_id':'product id'})
prod_cpg['cpg filter'] = prod_cpg['cpg_name']+" ("+prod_cpg['cpg_id'].astype(str)+")"
prod_cpg.sample(4)

,n,product id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name,cpg filter
89508,1,1846490,7501943491670,51,Pañales y toallitas,6903,KleenBebé,50,Kimberly Clark,Kimberly Clark (50)
72653,1,1071009,7506306207295,964,Lociones y jabones,143,Rexona,44,Unilever,Unilever (44)
24573,1,200972,41220710546,997,Barras energéticas y de granola,9992,HEB,1074,HEB,HEB (1074)
56969,1,368683,7501350987315,143,Helados y paletas,109,Nutrisa,25,Grupo Herdez,Grupo Herdez (25)


In [14]:
df = df.merge(prod_cpg[[('barcodes' if nielsen_cat else 'product id'),'cpg filter']],
              on=('barcodes' if nielsen_cat else 'product id'),how='left')

In [15]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
noviembre 2021,16293391.00,16293391.00,0.00,0.00


### Limpio

In [16]:
df = limpio(df,'month  ',
            ['month  ','country ','avg ticket','found rate','fulfillment','frequency'])
df.sample(2)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id,cpg_name
2776,Ciudad de México,Chedraui,Yogurt,24,369326,Yoghurt bebible sabor piña coco,7501032398606,Botella 220 g,nan,Danup (4349),...,BASE BUSINESS,nan,nan,Danone (75),11,2021,4349,Danup,75,Danone
6192,Ciudad Juárez,Soriana,Yogurt,24,379444,Yoghurt griego bebible sin azúcar añadida con ...,7501040095313,220 g,nan,Yoplait (6413),...,GRIEGO,nan,nan,Sigma Total (72),11,2021,6413,Yoplait,72,Sigma Total


In [17]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
nov,16293391.00,16293391.00,0.00,0.00


In [18]:
aux = df[df['formato'].astype(str)=='nan'].copy()
df = df[~(df['formato'].astype(str)=='nan')].copy()
null_fill = {'proveedor_general':'cpg_name',
             'proveedor_abierto':'cpg_name',
             'marca_detalle':'brand_name',
             'marca_generica':'brand_name',
             'gramaje':'package',
             'descripcion_nielsen':'product_name',
             'segmento':'category_en'}
for x,y in null_fill.items():
    aux.loc[:,x] = aux.loc[:,y]
    
df = df.append(aux,ignore_index=True)

In [19]:
for col in df.columns:
    df[col] = df[col].astype(str).str.replace('nan','')
    
df = df.drop(columns='cpg_name').fillna('')
df.sample()

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id
12921,Querétaro,Soriana,Yogurt,24,2934130,Yoghurt bebible triple cero fresa,7506443104617,Botella 220 g,444817,Vitalínea (17820),...,LIGHT,LIGHT,,,Danone (75),11,2021,17820,Vitalínea,75


In [20]:
brand_cols = ['formato','proveedor_general','proveedor_abierto','marca_detalle','marca_generica','gramaje','descripcion_nielsen','segmento','subsegmento_1','subsegmento_2','subsegmento_3']
display(df[brand_cols].sample())

for col in brand_cols:
    df[col] = df[col].str.strip().str.title()

display(df[brand_cols].sample())

,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
11708,INDIVIDUAL,SIGMA,SIGMA,YOPLAIT PLACER,YOPLAIT PLACER,0.145,YOPLAIT PLACER CRUNCH VAINILLA SOLIDO125GRC/CE...,SPECIALTIES,SPECIALTIES,,


,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
4956,Individual,Sigma,Sigma,Yoplait,Yoplait,0.22,Yoplait Yoghurt Natural Para Beber Regular 242...,Bb Liquido,Base Business,,


In [21]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux_fin

,mes,sales
0,nov,16293391.00


### Marca genérica

In [22]:
df.loc[df[df['marca_generica']==''].index, 'marca_generica'] = df.loc[df[df['marca_generica']==''].index, 'marca_detalle']
df[df['marca_generica']=='']

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id


## Export

### Same product_id as past

In [23]:
print(len(df))
if past:
    som = pd.ExcelFile(os.path.join('/'.join(BASE_DIR.split('/')[:-2]),
                                    f'SoM_{SOM_CPG}.xlsx')).parse(sheet_name=0)
    df = df.merge(som.drop_duplicates('product_id')[['product_id']].reset_index(drop=True).astype(str))
    
print(len(df))

17235
17235


In [24]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))

In [25]:
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1

### Ordered columns

In [26]:
export = df[['city_name', 'store_name', 'category_en', 'category_id', 'product_id',
             'product_name', 'barcodes', 'package', 'product_sku', 'brand_name',
             'sales', 'orders', 'users', 'units_found', 'units_replaced',
             'units_requested', 'formato', 'proveedor_general', 'proveedor_abierto',
             'marca_detalle', 'marca_generica', 'gramaje', 'descripcion_nielsen',
             'segmento', 'subsegmento_1', 'subsegmento_2', 'subsegmento_3',
             'mes', 'anio']].copy()
export.sort_values(['anio','mes'],inplace=True)
export.sample(4)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_name,...,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3,mes,anio
4785,Ciudad de México,Soriana,Yogurt,24,2934132,Yoghurt bebible sabor durazno,7506443104549,Botella 900 g,444813,Danone,...,Danone,Danone,0.9,Danone Sabor Durazno Bebible Bot 900 Gr,Bb Liquido,Base Business,,,11,2021
14822,Chihuahua,Fresh Market,Plant-based Beverages,1516,343185,Crema de coco,7503006575112,Cartón 1 L,,Calahua,...,Calahua,Calahua,Cartón 1 L,Crema De Coco,Plant-Based Beverages,,,,11,2021
10525,Puebla,Chedraui,Yogurt,24,1153213,Yoghurt batido zarzamora,7501055912476,125 g,7501055912476,Alpura,...,Alpura,Alpura,0.125,Alpura Yogurt Sabor Zarzamora Solido Vaso 125 Gr,Bb Solido,Base Business,,,11,2021
11471,Querétaro,Chedraui,Water,35,533631,Agua con toque sabor piña coco,758104007103,1.5 l,,Bonafont Levité,...,Levite,Levite,1.5,Levite Piã‘A Coco 1.5 Lt,Bebidas Saborizadas,Agua Saborizada,,,11,2021


In [27]:
export.to_csv(os.path.join(BASE_DIR,f'som_{CPG}.csv'),index=False,sep='\t',encoding='utf-16')

## Fin

In [28]:
aux

,sales,sales_fin,dif,dif_p
month,,,,
nov,16293391.00,16293391.00,0.00,0.00


In [29]:
time_exp(time.time() - start)
tono()

0 minutos con 3.38 segundos
